In [21]:
import pandas as pd
data = pd.read_excel('PRREPE.xlsx')

In [6]:
len(data['Latitude'])

15271

In [24]:
from joblib import Parallel, delayed
import requests
import pandas as pd

base_url = 'https://power.larc.nasa.gov/api/temporal/monthly/point'

# Function to retrieve temperature data for a given latitude and longitude
def retrieve_data(latitude, longitude):
    parameters = {
        "community": "RE",
        "longitude": longitude,
        "latitude": latitude,
        "start": "2001",
        "end": "2001",
        "parameters": "T2M_MIN,T2M_MAX,PRECTOTCORR_SUM,EVPTRNS,GWETPROF,WS2M_RANGE_AVG",
        "format": "JSON"
    }

    response = requests.get(base_url, params=parameters)

    if response.status_code == 200:
        data = response.json()
        if 'properties' in data and 'parameter' in data['properties']:
            parameter_data = data['properties']['parameter']
            if 'T2M_MAX' in parameter_data and 'T2M_MIN' in parameter_data and 'PRECTOTCORR_SUM' in parameter_data and 'EVPTRNS' in parameter_data and 'GWETPROF' in parameter_data and 'WS2M_RANGE_AVG' in parameter_data :
                
                df_max = pd.DataFrame(list(parameter_data['T2M_MAX'].items()), columns=['DateTime', 'T2M_MAX'])
                df_min = pd.DataFrame(list(parameter_data['T2M_MIN'].items()), columns=['DateTime', 'T2M_MIN'])
                df_prec = pd.DataFrame(list(parameter_data['PRECTOTCORR_SUM'].items()), columns=['DateTime','Precipitation'])
                df_evap = pd.DataFrame(list(parameter_data['EVPTRNS'].items()), columns=['Datetime','Evapotranspiration'])
                df_soil = pd.DataFrame(list(parameter_data['GWETPROF'].items()),columns=['Datetime','Soil_moisture'])
                df_wind = pd.DataFrame(list(parameter_data['WS2M_RANGE_AVG'].items()),columns=['Datetime','Wind_speed'])
                return df_max, df_min, df_prec, df_evap, df_soil, df_wind
    return None, None

# Coordinates
latitude_values = data['Latitude'].values[15270:15272]
longitude_values = data['Longitude'].values[15270:15272]

july_min = []
jan_max = []
prec = []
evap = []
soil = []
wind = []

# Parallelize the API requests and retrieve data
results = Parallel(n_jobs=8)(delayed(retrieve_data)(lat, lon) for lat, lon in zip(latitude_values, longitude_values))

# Process the results
for result_max, result_min, result_prec, result_evap, result_soil, result_wind in results:
    if result_max is not None and result_min is not None and result_prec is not None and result_evap is not None and result_soil is not None and result_wind is not None:
        # Process the temperature data as needed
        # Append to the respective lists (aurea_july_min, aurea_jan_max)
        df1 = result_max.loc[result_max['DateTime'] == '200101', 'T2M_MAX'].values[0]
        jan_max.append(df1)
        df2 = result_min.loc[result_max['DateTime'] == '200107', 'T2M_MIN'].values[0]
        july_min.append(df2)
        df3 = result_prec.loc[result_max['DateTime'] == '200113', 'Precipitation'].values[0]
        prec.append(df3)
        df4 = result_evap.loc[result_max['DateTime'] == '200113', 'Evapotranspiration'].values[0]
        evap.append(df4)
        df5 = result_soil.loc[result_max['DateTime'] == '200113', 'Soil_moisture'].values[0]
        soil.append(df5)
        df6 = result_wind.loc[result_max['DateTime'] == '200113', 'Wind_speed'].values[0]
        wind.append(df6)
        print("Temperature data retrieval completed.")

Temperature data retrieval completed.


In [17]:
cov = {'jan_max':jan_max, 'july_min':july_min, 'prec':prec, 'evap':evap, 'soil':soil, 'wind':wind}

In [18]:
dataf = pd.DataFrame(cov)
dataf

,jan_max,july_min,prec,evap,soil,wind
0,40.00,0.71,390.23,0.05,0.45,3.94
1,40.00,0.71,390.23,0.05,0.45,3.94
2,40.00,0.71,390.23,0.05,0.45,3.94
3,40.00,0.71,390.23,0.05,0.45,3.94
4,40.00,0.71,390.23,0.05,0.45,3.94
...,...,...,...,...,...,...
7266,36.73,1.57,458.79,0.16,0.45,3.73
7267,36.73,1.57,458.79,0.16,0.45,3.73
7268,36.73,1.57,458.79,0.16,0.45,3.73
7269,36.73,1.57,458.79,0.16,0.45,3.73


In [19]:
dataf.to_csv('PRREPE_cov.csv',index=False)